In [1]:
# Get Entities
def get_entities_update(sent):
    
    ent1 = ""
    ent2 = ""
    
    
    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence
    
    prefix = ""
    modifier = ""
    
    list_ = []
    
    for tok in nlp(sent):
        if tok.dep_ != "punct":
            if tok.dep_ == "compound":
                prefix = tok.text
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " "+ tok.text
    
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " "+ tok.text
    
            if tok.dep_.find("subj") == True:
                #ent1 = modifier +" "+ prefix + " "+ tok.text
                ent = ''
                ent = modifier +" "+ prefix + " "+ tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""
                list_.append(ent)
    
            if tok.dep_.find("obj") == True:
                ent = ''
                ent = modifier +" "+ prefix +" "+ tok.text
                list_.append(ent)
    
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text
    #return [ent1.strip(), ent2.strip()]
    return list_

def RemoveStopWords(List_):
    #Single_Word_Tokens = []
    List_without_space = [x.strip() for x in List_]
    for key in List_without_space:
        Token = ''.join(key).strip()
        Number_of_words = len(Token.split(" "))
    
    #print(key)
        if Number_of_words== 1:
            #Single_Word_Tokens.append(Token)
            doc = nlp(Token) # Convert to Spacy Object
            for token in doc:
                if token.is_stop:
                    token =str(token) # Convert from Spacy to String for list operations
                    #print(token)
                    try:
                        List_without_space.remove(token)
                    except:
                        pass
                        
                
    return List_without_space 

In [2]:
import os
import json
import re
import spacy
import pandas as pd
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import re
import pickle



path=r'/opt/ml-training/Notebooks/Sahil/KnowledgeGraph/DataSet/Case_Study'
print(path)

/opt/ml-training/Notebooks/Sahil/KnowledgeGraph/DataSet/Case_Study


In [3]:
pwd()

'/opt/ml-training/Notebooks/Sahil/KnowledgeGraph/SP_10_Noun_Mod_Phrases/Case_Study'

In [4]:
List_of_Values = []
counter = 0
for i in os.walk(path):
    #print(i)
    for j in i[2]:
        
        StringVal = ''
        List_of_Sentences = []
        file = open(path+'/'+j, encoding="utf8")
        #print(file)
        data=json.loads(file.read())
        StringVal = data['body_t']
        #List_of_Values.append(get_entities_Noun(StringVal))
        List_of_Sentences = StringVal.split("\n")
        List_of_Sentences = [re.sub('[^a-zA-Z0-9]+'," ",x) for x in List_of_Sentences ]
        counter  = counter + 1
        print(counter)
        for each in List_of_Sentences:
        #SearchString = str(each) #  = ' 49 EDGE Project Mgmt Work Mgmt INREV Initiate Project Reviews PM10 Project Mgmt Work Mgmt '
            if len(each) >1:
                List_of_Values.append(get_entities_update(each))
            else:
                pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [5]:
print("List of Values Length:",len(List_of_Values))
#Flatten the List
List_consolidated = [val for each in List_of_Values for val in each]
print("List_consolidated Length:",len(List_consolidated))
# Save Original File
with open('CaseStudy.pkl', 'wb') as f:
    pickle.dump(List_consolidated, f)

List of Values Length: 58204
List_consolidated Length: 318467


In [6]:
word_freq = Counter(List_consolidated)
common_words = word_freq.most_common(1000)
print (common_words)

[('  you', 5981), ('  it', 4971), ('  that', 4563), ('  we', 4501), ('  I', 4121), ('  t', 1682), ('  what', 1504), ('  this', 1400), ('  they', 1357), ('  We', 1222), (' N ERROR', 1042), ('  us', 1030), ('  It', 966), ('  them', 903), ('  You', 758), ('  which', 747), ('  me', 620), ('  data', 602), ('So  we', 581), ('  all', 565), ('So  I', 561), ('  time', 533), ('  site', 508), ('  people', 507), ('So  you', 485), ('  lot', 479), ('  things', 476), ('  one', 458), (' czon obs', 454), ('  the', 434), ('  some', 380), ('  s', 375), ('  process', 368), ('  That', 366), ('  information', 365), ('how  we', 356), ('where  we', 349), ('  business', 348), ('  who', 337), ('  What', 327), ('  use', 325), ('  This', 318), ('  way', 318), ('personal  data', 313), ('  customers', 307), ('  They', 299), ('  m', 297), ('  company', 295), ('So  it', 284), ('0  0', 281), ('  questions', 277), ('  he', 269), ('  something', 265), ('So  that', 264), ('  DXC', 263), ('  work', 252), ('  order', 251),

In [9]:
# Remove Stopwords
ConsoladatedList= []
ConsoladatedList = RemoveStopWords(List_consolidated)
len(ConsoladatedList)

285159

In [10]:
# Save StopWords File
with open('CaseStudy_SW.pkl', 'wb') as f:
    pickle.dump(ConsoladatedList, f)

In [7]:
with open('CaseStudy_SW.pkl', 'rb') as f:
    ConsoladatedList = pickle.load(f)
len(ConsoladatedList)

285159

In [8]:
# Remove Single Characters tokens
[ConsoladatedList.remove(each) for each in ConsoladatedList if len(each.strip()) ==1]
len(ConsoladatedList)

281246

In [9]:
ConsoladatedList = [re.sub('\S+@\S+'," ",x) for x in ConsoladatedList ] # Replace EmailAddress
ConsoladatedList = [re.sub("e.g. "," ",x) for x in ConsoladatedList]
len(ConsoladatedList)

281246

In [11]:
List_Of_Removals = [' ', '', ' ', 'Confidential Proprietary', 'Proprietary', 'Confidential', 'Confidential Proprietary', 
                    'Don t','DXC Proprietary', 'Status e','g Release', 'AM117100534 Assessment', 'N A']
[ConsoladatedList.remove(each) for each in ConsoladatedList if each in List_Of_Removals]
len(ConsoladatedList)

281112

In [15]:
for x in list(ConsoladatedList):
    token = x.split(" ")
    if len(token) == 2:
        if token[0] == token[1]:
            ConsoladatedList.remove(x)

len(ConsoladatedList)

280974

In [18]:
word_freq = Counter(ConsoladatedList)
common_words = word_freq.most_common(1050)
print (common_words)

[('you', 1587), ('we', 1361), ('it', 1277), ('N ERROR', 1042), ('that', 785), ('data', 602), ('So  we', 581), ('So  I', 561), ('time', 533), ('site', 508), ('people', 507), ('So  you', 485), ('lot', 479), ('things', 476), ('czon obs', 454), ('process', 368), ('what', 368), ('information', 365), ('they', 363), ('how  we', 356), ('where  we', 349), ('business', 348), ('use', 325), ('this', 321), ('way', 318), ('personal  data', 313), ('customers', 307), ('company', 295), ('We', 291), ('So  it', 284), ('0  0', 281), ('questions', 277), ('So  that', 264), ('DXC', 263), ('work', 252), ('order', 251), ('cookies', 248), ('us', 241), ('more  You', 238), ('so  I', 235), ('however  they', 233), ('automated data We', 232), ('browsing experience', 232), ('however  cases', 232), ('properly  more', 232), ('properly cookies privacy', 232), ('properly cookies website', 232), ('properly Online Privacy Notice', 232), ('properly Online Privacy Use', 232), ('service', 228), ('Masz', 227), ('uguje lub Twoj

In [24]:
Two_Tokens_Removals = ['you','now', 'we', 'it', 'that', 'so', 'what', 'this', 'they','us', 'more', 'however', 'when', 'where', 
                    'then', 'them', 'just', 'll', 'here', 'other', 'which', 'why', 'actually', 'other', 'their', 'how',
                     'really', 'there', 'again']

for each in list(ConsoladatedList):
    token = each.split(" ")
    if token[0].lower() in Two_Tokens_Removals:
        ConsoladatedList.remove(each)

len(ConsoladatedList)

237983

In [26]:
word_freq = Counter(ConsoladatedList)
common_words = word_freq.most_common(1050)
print (common_words)

[('data', 602), ('time', 533), ('N ERROR', 520), ('site', 508), ('people', 507), ('lot', 479), ('things', 476), ('czon obs', 454), ('process', 368), ('information', 365), ('business', 348), ('personal  data', 313), ('customers', 307), ('company', 295), ('0  0', 281), ('questions', 277), ('DXC', 263), ('work', 252), ('order', 251), ('cookies', 248), ('automated data We', 232), ('browsing experience', 232), ('properly  more', 232), ('properly cookies privacy', 232), ('properly cookies website', 232), ('properly Online Privacy Notice', 232), ('properly Online Privacy Use', 232), ('service', 228), ('Masz', 227), ('uguje lub Twoja Je', 227), ('masz', 227), ('czon JavaScript', 227), ('czy aby poprawnie', 227), ('wietla', 227), ('t i', 227), ('korzysta', 227), ('wszystkich jej', 227), ('ci', 227), ('Strategy Consulting Engineering', 227), ('Expertise Automotive Energy', 227), ('Digital Capabilities', 227), ('Telecommunications Travel Strategy Consulting Scale', 226), ('Telecommunications Trav

In [22]:
List_Of_Removals = ['N ERROR', 'me', 'use', 'way', '0 N ERROR', 'ERROR N ERROR', 'lot', 't i', 'example', 'one  I']
[ConsoladatedList.remove(each) for each in ConsoladatedList if each in List_Of_Removals]
len(ConsoladatedList)

241703

In [23]:
word_freq = Counter(ConsoladatedList)
common_words = word_freq.most_common(1050)
print (common_words)

[('data', 602), ('time', 533), ('N ERROR', 520), ('site', 508), ('people', 507), ('lot', 479), ('things', 476), ('czon obs', 454), ('process', 368), ('information', 365), ('business', 348), ('personal  data', 313), ('customers', 307), ('company', 295), ('0  0', 281), ('questions', 277), ('DXC', 263), ('work', 252), ('order', 251), ('cookies', 248), ('automated data We', 232), ('browsing experience', 232), ('properly  more', 232), ('properly cookies privacy', 232), ('properly cookies website', 232), ('properly Online Privacy Notice', 232), ('properly Online Privacy Use', 232), ('service', 228), ('Masz', 227), ('uguje lub Twoja Je', 227), ('masz', 227), ('czon JavaScript', 227), ('czy aby poprawnie', 227), ('wietla', 227), ('t i', 227), ('korzysta', 227), ('wszystkich jej', 227), ('ci', 227), ('Strategy Consulting Engineering', 227), ('Expertise Automotive Energy', 227), ('Digital Capabilities', 227), ('Telecommunications Travel Strategy Consulting Scale', 226), ('Telecommunications Trav

In [36]:
## Capture Tags with Length greater than 1
Len_Greater1 = []
for each in list(ConsoladatedList):
    token = each.split(" ")
    if len(token) >1:
        Len_Greater1.append(each)
len(Len_Greater1)

179823

In [44]:
word_freq = Counter(Len_Greater1)
common_words = word_freq.most_common(1050)
print (common_words)

[('N ERROR', 520), ('czon obs', 454), ('personal  data', 313), ('0  0', 281), ('automated data We', 232), ('browsing experience', 232), ('properly  more', 232), ('properly cookies privacy', 232), ('properly cookies website', 232), ('properly Online Privacy Notice', 232), ('properly Online Privacy Use', 232), ('uguje lub Twoja Je', 227), ('czon JavaScript', 227), ('czy aby poprawnie', 227), ('t i', 227), ('wszystkich jej', 227), ('Strategy Consulting Engineering', 227), ('Expertise Automotive Energy', 227), ('Digital Capabilities', 227), ('Telecommunications Travel Strategy Consulting Scale', 226), ('Telecommunications Travel Insights Newsroom Careers', 226), ('rights Quality policy Terms', 226), ('copyright violation IMPORTANT', 226), ('copyright violation page', 226), ('Telecommunications Travel Strategy Technologies', 225), ('S OP process', 168), ('S OP', 149), ('business case', 136), ('Program process', 116), ('0 Computer Sciences Corporation', 115), ('0 Computer Sciences rights', 1

In [41]:
for x in Len_Greater1:
    token = x.split(" ")
    if len(token) == 2:
        if token[0] == token[1]:
            Len_Greater1.remove(x)

len(Len_Greater1)

179823

In [43]:
Two_Tokens_Removals = ['you','now', 'we', 'it', 'that', 'so', 'what', 'this', 'they','us', 'more', 'however', 'when', 'where', 
                    'then', 'them', 'just', 'll', 'here', 'other', 'which', 'why', 'actually', 'other', 'their', 'how',
                     'really', 'there', 'again']

for each in list(Len_Greater1):
    token = each.split(" ")
    if token[0].lower() in Two_Tokens_Removals:
        Len_Greater1.remove(each)

len(Len_Greater1)

179823

In [29]:
# Save to CSV

List_Phrase = [phrase for phrase,frequency in common_words]
List_Frequency = [frequency for phrase,frequency in common_words]

print(len(List_Phrase))
print(len(List_Frequency))

#for k,v in common_words:
#    print(v)
#import pandas as pd

DF = pd.DataFrame()
DF['Phrases'] = List_Phrase
DF['Frequency'] = List_Frequency

DF.to_excel("Client_win_OnlyNoun.xlsx")

1050
1050
